[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gabays/32M7131/blob/main/Cours_03/Cours03.ipynb)

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licence Creative Commons" style="border-width:0;float:right;\" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>

Distant Reading 2: linguistique computationnelle

# **Lemmatiser un texte**

Simon Gabay



🚨 Pour les entraînements, on va avoir besoin d'un GPU. Si vous utilisez Colab, on va en demander un à notre meilleur ami Google un en allant sur dans le menu en haut à gauche et en choisissant `Execution` > `Modifier le type d'exécution` puis en choisissant `GPU`.

⚠️ <font color='red'>Attention! l'usage des GPU est limité sur Google! Il faut les utiliser avec parcimonie, sinon il faut payer! Ou bien vous pouvez tout faire en ligne de commande sur les clusters HPC de l'uni.</font>

## 0. Préparer l'environnement de travail

Vous devez définir le dossier de travail, qui n'est pas le même sur Colab ou OpenOnDemand.
- si vous êtes sur Google colab il s'agit de `/content`

In [1]:
DIRECTORY="/content"

- Si vous êtes sur OpenOnDemand, créez un dossier et définissez une variable avec le chemin vers ce fichier

In [2]:
#On crée un dossier où on va mettre les documents
!mkdir -p Lemmatisation
#On définit une variable avec le chemin vers ce dossier
DIRECTORY="Lemmatisation"

On va nettoyer notre env des trucs inutiles (installés préalablement).
Note: c'est inutile sur Colab.

In [3]:
!pip freeze --user --exclude-editable | xargs pip uninstall -y
!pip list

Found existing installation: gensim 4.3.2
Uninstalling gensim-4.3.2:
  Successfully uninstalled gensim-4.3.2
Found existing installation: JSON-minify 0.3.0
Uninstalling JSON-minify-0.3.0:
  Successfully uninstalled JSON-minify-0.3.0
Found existing installation: lxml 5.1.0
Uninstalling lxml-5.1.0:
  Successfully uninstalled lxml-5.1.0
Found existing installation: nlp-pie 0.3.8
Uninstalling nlp-pie-0.3.8:
  Successfully uninstalled nlp-pie-0.3.8
Found existing installation: numpy 1.17.5
Uninstalling numpy-1.17.5:
  Successfully uninstalled numpy-1.17.5
Found existing installation: PyYAML 5.1b3
Uninstalling PyYAML-5.1b3:
  Successfully uninstalled PyYAML-5.1b3
Found existing installation: scikit-learn 0.22.2.post1
Uninstalling scikit-learn-0.22.2.post1:
  Successfully uninstalled scikit-learn-0.22.2.post1
Found existing installation: scipy 1.10.1
Uninstalling scipy-1.10.1:
  Successfully uninstalled scipy-1.10.1
Found existing installation: smart-open 7.0.1
Uninstalling smart-open-7.0.1:


## 1. Annoter

### 1.1 Premier test

Il nous faut d'abord installer [_pie-extended_](https://pypi.org/project/pie-extended/) pour utiliser le lemmatiseur [_Pie_](https://github.com/emanjavacas/pie). Il existe bien d'autres outils: l'un des plus populaires est [_spaCy_](https://spacy.io), qui couvre les principales langues (allemand, français, anglais, espagnol, anglais…), mais nous nous intéressons à des langues moins bien dotées et d'une nature différente (pré-orthographiques) qui nécessitent des solutions spécifiques.

In [4]:
!pip install --upgrade pip --user
!pip install pie-extended --user

  Using cached pip-24.0-py3-none-any.whl (2.1 MB)
  Using cached pie_extended-0.1.2-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached PaPie-0.4.2-py2.py3-none-any.whl.metadata (16 kB)
  Using cached autodisambiguator-0.0.1-py2.py3-none-any.whl.metadata (817 bytes)
  Using cached requests-2.25.1-py2.py3-none-any.whl.metadata (4.2 kB)
  Using cached Unidecode-1.1.2-py2.py3-none-any.whl.metadata (12 kB)
  Using cached JSON_minify-0.3.0-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached pytorch_optimizer-2.7.0-py3-none-any.whl.metadata (32 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached terminaltables-3.1.10-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
  Using cached typing-3.7.4.3-py3-none-any.whl
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using

Nous téléchargeons les modèles pour le français (dit `fr`) -- une liste des modèles disponibles est [en ligne](https://pypi.org/project/pie-extended).

In [5]:
!pie-extended download fr

Starting downloading...
8 files to download
[██████████████████████████████████████████████████]
- lemma.tar downloaded
[██████████████████████████████████████████████████]
- pos.tar downloaded
[██████████████████████████████████████████████████]
- mode.tar downloaded
[██████████████████████████████████████████████████]
- temps.tar downloaded
[██████████████████████████████████████████████████]
- pers.tar downloaded
[██████████████████████████████████████████████████]
- nomb.tar downloaded
[██████████████████████████████████████████████████]
- genre.tar downloaded
[██████████████████████████████████████████████████]
- cas.tar downloaded
Finished !


Nous créons un fichier `essai.txt` un avec la phrase _je m'appelle Simon_:

In [6]:
!echo "je m'appelle Simon" >> $DIRECTORY/essai.txt

Avec `pie extended` j'annote (`tag`) avec le modèle `fr`:

In [7]:
!pie-extended tag fr $DIRECTORY/essai.txt

Getting the tagger
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  3.35it/s]


Il me crée automatiquement un fichier du même nom en ajoutant `-pie` au nom avant l'extension. Je peux regarder le résultat en ouvrant le fichier avec la commande bash `cat`:

In [8]:
!cat $DIRECTORY/essai-pie.txt

token	lemma	POS	morph	treated
je	je	PROper	PERS.=1|NOMB.=s|CAS=n	je
m'	je	PROper	PERS.=1|NOMB.=s|CAS=r	m'
appelle	appeler	VERcjg	MODE=ind|TEMPS=pst|PERS.=1|NOMB.=s	appelle
Simon	Simon	NOMpro	NOMB.=s	Simon
je	je	PROper	PERS.=1|NOMB.=s|CAS=n	je
m'	je	PROper	PERS.=1|NOMB.=s|CAS=r	m'
appelle	appeler	VERcjg	MODE=ind|TEMPS=pst|PERS.=1|NOMB.=s	appelle
Simon	Simon	ADVgen	NOMB.=s	Simon
je	je	PROper	PERS.=1|NOMB.=s|CAS=n	je
m'	je	PROper	PERS.=1|NOMB.=s|CAS=r	m'
appelle	appeler	VERcjg	MODE=ind|TEMPS=pst|PERS.=1|NOMB.=s	appelle
Simon	Simon	ADVgen	NOMB.=s	Simon
je	je	PROper	PERS.=1|NOMB.=s|CAS=n	je
m'	je	PROper	PERS.=1|NOMB.=s|CAS=r	m'
appelle	appeler	VERcjg	MODE=ind|TEMPS=pst|PERS.=1|NOMB.=s	appelle
Simon	Simon	NOMpro	NOMB.=s	Simon
je	je	PROper	PERS.=1|NOMB.=s|CAS=n	je
m'	je	PROper	PERS.=1|NOMB.=s|CAS=r	m'
appelle	appeler	VERcjg	MODE=ind|TEMPS=pst|PERS.=1|NOMB.=s	appelle
Simon	Simon	NOMpro	NOMB.=s	Simon


### 1.2 Un test plus ambitieux

Nous recommençons avec un texte plus complexe: la première fable de _L’École des femmes_ de Molière [disponible sur wikisource]([Wikisource](https://fr.wikisource.org/wiki/L%E2%80%99%C3%89cole_des_femmes/%C3%89dition_Chasles,_1888):

In [9]:
!curl https://raw.githubusercontent.com/gabays/32M7131/master/Cours_03/docs/moliere.txt > $DIRECTORY/moliere.txt
!head -10 $DIRECTORY/moliere.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10246  100 10246    0     0  49871      0 --:--:-- --:--:-- --:--:-- 50722
Scène I
Chrysalde, Arnolphe
chrysalde.
Vous venez, dites-vous, pour lui donner la main ?
arnolphe.
Oui. Je veux terminer la chose dans demain.
chrysalde.
Nous sommes ici seuls ; et l’on peut, ce me semble,
Sans craindre d’être ouïs, y discourir ensemble.
Voulez-vous qu’en ami je vous ouvre mon cœur ?


J'annote ce fichier:

In [10]:
!pie-extended tag fr $DIRECTORY/moliere.txt
!head -10 $DIRECTORY/moliere-pie.txt

Getting the tagger
100%|█████████████████████████████████████████████| 1/1 [00:09<00:00,  9.16s/it]
token	lemma	POS	morph	treated
Scène	scène	NOMcom	NOMB.=s|GENRE=f	Scène
I	I	PROper	NOMB.=s|GENRE=f	I
Chrysalde	Chrysalde	NOMpro	NOMB.=s|GENRE=f	Chrysalde
,	,	PONfbl	MORPH=empty	,
Arnolphe	Arnolphe	NOMpro	NOMB.=s|GENRE=m	Arnolphe
chrysalde	chrysalde	NOMpro	NOMB.=s|GENRE=f	chrysalde
.	.	PONfrt	MORPH=empty	.
Vous	vous	PROper	PERS.=2|NOMB.=p	Vous
venez	venir	VERcjg	MODE=ind|TEMPS=pst|PERS.=2|NOMB.=p	venez


### 1.3 Un essai en latin
 Recommençons avec un texte en latin. Il faut d'abord charger le modèle latin et les _addons_

In [11]:
!pie-extended download lasla
!pie-extended install-addons lasla

Starting downloading...
12 files to download
[████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████]
- latin-straight.json downloaded
[████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████]
- latin-pos.json downloaded
[██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████]
- latin-needs.json downloaded
[██████████████████████████████████████████████████]
- Mood_Tense_Voice.tar downloaded
[██████████████████████████████████████████████████]
- Gend.tar downloaded
[██████████████████████████████████████████████████

Téléchargeons désormais un texte, le _Pro Quinctio_ de Cicéron disponible sur [_The Latin Library_](https://www.thelatinlibrary.com/cicero/quinc.shtml).

In [12]:
!curl https://raw.githubusercontent.com/gabays/32M7131/master/Cours_03/docs/Cicero_%20Pro_Quinctio.txt > $DIRECTORY/Cicero_%20Pro_Quinctio.txt
!head -10 $DIRECTORY/Cicero_%20Pro_Quinctio.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 58714  100 58714    0     0   255k      0 --:--:-- --:--:-- --:--:--  258k
M. TVLLI CICERONIS PRO P. QVINCTIO ORATIO

I. Quae res in civitate duae plurimum possunt, eae contra nos ambae faciunt in hoc tempore, summa gratia et eloquentia; quarum alterum, C. Aquili, vereor, alteram metuo. Eloquentia Q. Hortensi ne me in dicendo impediat, non nihil commoveor, gratia Sex. Naevi ne P. Quinctio noceat, id vero non mediocriter pertimesco. Neque hoc tanto opere querendum videretur, haec summa in illis esse, si in nobis essent saltem mediocria; verum ita se res habet, ut ego, qui neque usu satis et ingenio parum possum, cum patrono disertissimo comparer, P. Quinctius, cui tenues opes, nullae facultates, exiguae amicorum copiae sunt, cum adversario gratiosissimo contendat. Illud quoque nobis accedit incommodum, quod M. Iunius, qui hanc 

On annote et on observe le résultat:

In [13]:
!pie-extended tag lasla $DIRECTORY/Cicero_%20Pro_Quinctio.txt
!head -10 $DIRECTORY/Cicero_%20Pro_Quinctio-pie.txt

Getting the tagger
100%|█████████████████████████████████████████████| 1/1 [00:47<00:00, 47.42s/it]
token	lemma	pos	morph	Dis	treated
M.	Marcus	NOMpro	Case=Gen|Numb=Sing	_	M
TVLLI	Tullius	NOMpro	Case=Gen|Numb=Sing	_	TULLI
CICERONIS	Ciceroni	NOMpro	Case=Nom|Numb=Sing	_	CICERONIS
PRO	Pro	PRE	MORPH=empty	1	PRO
P.	Publius	NOMpro	Case=Abl|Numb=Sing	_	P
QVINCTIO	Quintius	NOMpro	Case=Abl|Numb=Sing|Person=3	_	QUINCTIO
ORATIO	Oratius	NOMpro	Case=Abl|Numb=Sing	_	ORATIO
I	1	ADJcar	MORPH=empty	_	1
.	.	PUNC	MORPH=empty	_	--IGN.--


## 2. Entraîner un modèle
### 2.1 _Split_ pour l'entraînement

Nous allons d'abord faire un _split_ et créer les trois jeux de données nécessaires avec [_Protogenie_](https://github.com/hipster-philology/protogenie).

Récupérons d'abord un jeu de données.

In [14]:
!curl https://raw.githubusercontent.com/gabays/32M7131/main/Cours_03/train/lemma_Empty.txt > $DIRECTORY/lemma_Empty.txt
!head -20 $DIRECTORY/lemma_Empty.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1917k  100 1917k    0     0  6428k      0 --:--:-- --:--:-- --:--:-- 6501k
form	lemma
1536_Flores_Deplourable.tsv	xxx
form	lemma
GRIMALTE	Grimalte
AMANT	amant
DE	de
LA	le
dame	dame
Gradisse	Gradisse
narre	narrer
sommairement	sommairement
Les	le
amoureux	amoureux
regredz	regret
de	de
Flamete	Flammette
,	,
qui	qui
furent	être
occasion	occasion


Ensuite nous installons _Protogenie_.

In [15]:
!pip install protogenie --user

  Using cached protogenie-0.0.7-py2.py3-none-any.whl.metadata (2.4 kB)
Using cached protogenie-0.0.7-py2.py3-none-any.whl (23 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 39.4 MB/s eta 0:00:0000:0100:01


On charge un fichier de config (pour apprendre à les fabriquer, il existe une [documentation](https://github.com/hipster-philology/protogenie/blob/master/DOCUMENTATION.md)).



In [16]:
!curl https://raw.githubusercontent.com/gabays/32M7131/main/Cours_03/train/lemma_Empty.xml > $DIRECTORY/lemma_Empty.xml
!cat $DIRECTORY/lemma_Empty.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   570  100   570    0     0   3049      0 --:--:-- --:--:-- --:--:--  3114
<?xml version="1.0" encoding="UTF-8"?>
<?xml-model href="protogeneia/schema.rng" schematypens="http://relaxng.org/ns/structure/1.0"?>
<config>
    <default-header>
        <header type="explicit">
            <key>form</key>
            <key>lemma</key>
        </header>
    </default-header>
    <output column_marker="TAB">
        <header name="default"/>
    </output>
    <corpora>
        <corpus path="lemma_Empty.txt" column_marker="TAB">
            <splitter name="empty_line"/>
            <header type="default"/>
        </corpus>
    </corpora>
</config>


On fait le split -- ici 10% en `dev`, 10% en `eval` (ou `test`) et 80% en `train`, mais cette répartition peut changer en fonction de la quantité de données dont on dispose.

In [17]:
!protogenie build $DIRECTORY/lemma_Empty.xml -d 0.1 -e 0.1 -t 0.8

/home/users/g/gabays/.local/lib/python3.8/site-packages/protogenie/configs.py:157: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not header_node or header_node.get("name", "default") == "default":
Processing...
/home/users/g/gabays/Lemmatisation/lemma_Empty.txt has been transformed
	20221 tokens in test dataset
	19128 tokens in dev dataset
	153746 tokens in train dataset


Sur Colab, il est possible qu'il faille changer la valeur de l'attribut `path` dans `lemma_Empty.xml`:
```xml
<corpus path="lemma_Empty.txt" column_marker="TAB">
```
en 
```xml
<corpus path="/content/lemma_Empty.txt" column_marker="TAB">
```

### 2.2 Entrainement
On va entraîner avec [_Pie_](https://github.com/emanjavacas/pie).

In [18]:
!pip freeze --user --exclude-editable | xargs pip uninstall -y
!pip install nlp-pie --user

Found existing installation: autodisambiguator 0.0.1
Uninstalling autodisambiguator-0.0.1:
  Successfully uninstalled autodisambiguator-0.0.1
Found existing installation: colorama 0.4.6
Uninstalling colorama-0.4.6:
  Successfully uninstalled colorama-0.4.6
Found existing installation: JSON-minify 0.3.0
Uninstalling JSON-minify-0.3.0:
  Successfully uninstalled JSON-minify-0.3.0
Found existing installation: lightning-utilities 0.11.1
Uninstalling lightning-utilities-0.11.1:
  Successfully uninstalled lightning-utilities-0.11.1
Found existing installation: lxml 5.1.0
Uninstalling lxml-5.1.0:
  Successfully uninstalled lxml-5.1.0
Found existing installation: numpy 1.23.5
Uninstalling numpy-1.23.5:
  Successfully uninstalled numpy-1.23.5
Found existing installation: nvidia-cublas-cu11 11.10.3.66
Uninstalling nvidia-cublas-cu11-11.10.3.66:
  Successfully uninstalled nvidia-cublas-cu11-11.10.3.66
Found existing installation: nvidia-cuda-nvrtc-cu11 11.7.99
Uninstalling nvidia-cuda-nvrtc-cu11-

On a besoin de renommer les fichiers créés avec _Protogénie_ en `.tsv`

In [19]:
!mv output/dev/lemma_Empty.txt output/dev/lemma_Empty.tsv
!mv output/test/lemma_Empty.txt output/test/lemma_Empty.tsv
!mv output/train/lemma_Empty.txt output/train/lemma_Empty.tsv
!mv output $DIRECTORY

mv: cannot move 'output' to 'Lemmatisation/output': File exists


Sur colab: rajouter `/content/` devant `output/`.

Nous avons aussi besoin d'un fichier de configuration

In [20]:
!curl https://raw.githubusercontent.com/gabays/32M7131/main/Cours_03/train/config.json > $DIRECTORY/config.json
!cat $DIRECTORY/config.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1952  100  1952    0     0   9932      0 --:--:-- --:--:-- --:--:-- 10061
{
   "modelname":"mymodel",
   "modelpath":"./",
   "run_test":false,
   "input_path":"Lemmatisation/output/train/lemma_Empty.tsv",
   "test_path":"Lemmatisation/output/test/lemma_Empty.tsv",
   "dev_path":"Lemmatisation/output/dev/lemma_Empty.tsv",
   "header":true,
   "sep":"\t",
   "breakline_ref":"lemma",
   "breakline_data":"NONE",
   "char_max_size":500,
   "word_max_size":20000,
   "max_sent_len":35,
   "max_sents":1000000,
   "char_min_freq":1,
   "word_min_freq":1,
   "char_eos":true,
   "char_bos":true,
   "char_lower":false,
   "tasks":[
      {
         "name":"lemma",
         "target":true,
         "context":"sentence",
         "level":"char",
         "decoder":"attentional",
         "settings":{
            "bos":true,
            "eo

Ce fichier contient toutes les informations nécessaires pour l'entraînement. Le paramétrage via le fichier de configuration est extrêmement complexe, et il est difficile (impossible?) de connaître à l'avance les meilleurs choix à opérer: il faut souvent faire de nombreux tests.

Sur colab: changer la valeur de `input_path`, `test_path` et `dev_path` en remplaçant `Lemmatisation` par `/content`.

Si vous voulez un peu comprendre le contenu, voici quelques explications:
```json
{
  // * General
  // model name to be used for saving
  "modelname": "latest-fro",
  // model path to be used for saving
  "modelpath": "./",
  // run test (no serialization)
  "run_test": false,
  // max length of sentences (longer sentence will be split)
  "max_sent_len": 35,
  // maximum number of sentences to process
  "max_sents": 1000000,
  // * Data
  // path or unix-like expression to file(s)/dir with training data:
  // e.g. "datasets/capitula_classic/train0.tsv""
  "input_path": "corpus_for_train/train/CORPUS.tsv",
  // path to test set (same format as input_path)
  "test_path": "corpus_for_train/test/CORPUS.tsv",
  // path to dev set (same format as input_path)
  "dev_path": "corpus_for_train/dev/CORPUS.tsv",
  // data to use as reference for breaking lines (e.g. "pos")
  "breakline_ref": "POS",
  // needed to decide for a sentence boundary (e.g. "$.")
  "breakline_data": "INUTILE",
  // maximum vocabulary size
  "char_max_size": 500,
  // maximum vocabulary size for word input
  "word_max_size": 20000,
  // min freq per item to be incorporated in the vocabulary (only used if *_max_size is 0)
  "char_min_freq": 1,
  "word_min_freq": 1,
  // char-level encoding
  "char_eos": true,
  "char_bos": true,
  // tab-format only:
  "header": true,
  // separator for csv-like files
  "sep": "\t",
  // task-related config
  "tasks": [
    // each task's name refers to the corresponding data field
    // this behaviour can be changed in case the name differs from the data field
    // by using a "target" key that refers to the target data field
    // e.g. {"name": "lemma-char", "settings": {"target": "lemma"}}
    // e.g. {"name": "lemma-word", "settings": {"target": "lemma"}}
    {
      "name": "lemma",
      "target": true,
      "context": "sentence",
      "level": "char",
      "decoder": "attentional",
      "settings": {
        "bos": true,
        "eos": true,
        "lower": true,
        "target": "lemma"
      },
      "layer": -1
    },
    {"name": "POS"}
  ],
  "task_defaults": {
    "level": "token",
    "layer": -1,
    "decoder": "linear",
    "context": "sentence"
  },
  // general task schedule params (can be overwritten in the "settings" entry of each)
  "patience": 1000000, // default to very large value
  "factor": 1, // decrease the loss weight by this amount (0, 1)
  "threshold": 0, // minimum decrease in loss to be considered an improvement
  "min_weight": 0, // minimum value a task weight can be decreased to

  // whether to include autoregressive loss
  "include_lm": true,
  // whether to share the output layer for both fwd and bwd lm
  "lm_shared_softmax": false,
  // lm settings in case it's included as an extra task
  "lm_schedule": {
    "patience": 100, "factor": 0.5, "weight": 0.2, "mode": "min"
  }
}
  ```



Et on entraine. Le nombre d'itérations (_epochs_) est volontairement limité à 10 pour ne pas gaspiller trop d'énergie inutilement. En augmentant ce nombre, on obtiendrait un bien meilleur résultat.

Vous allez voir apparaître les scores sur le `dev` our chaque itération. Pour lire les résultats, [cf. précision et rappel sur wikipedia](https://fr.wikipedia.org/wiki/Pr%C3%A9cision_et_rappel).

In [21]:
!pip uninstall typing --yes
!pie train $DIRECTORY/config.json

Found existing installation: typing 3.7.4.3
Uninstalling typing-3.7.4.3:
  Successfully uninstalled typing-3.7.4.3
fatal: not a git repository (or any parent up to mount point /)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
It seems like you downloaded `pie` instead of git-cloning it or installing it with pip.
We won't be able to check compatibility between pretrained models and `pie` version.


::: Loaded Config :::

batch_size: 128
breakline_data: NONE
breakline_ref: lemma
buffer_size: 10000
cache_dataset: true
cell: GRU
cemb_dim: 400
cemb_layers: 2
cemb_type: rnn
char_bos: true
char_eos: true
char_lower: false
char_max_size: 500
char_min_freq: 1
checks_per_epoch: 1
clip_norm: 5
config_path: Lemmatisation/config.json
custom_cemb_cell: false
decoder_layers: 3
dev_path: Lemmatisation/output/dev/lemma_Empty.tsv
device: cuda
drop_diacritics: false
dropout: 0.25
epochs: 10
factor: 0.5
freeze_embeddings: false
header: true
hidden_size: 256
include_lm: true
ini

In [26]:
!mv mymodel-lemma-*.tar mymodel.tar

On peut désormais utiliser ce modèle avec _Pie_:


In [23]:
! pie tag $DIRECTORY/moliere.txt mymodel.tar

fatal: not a git repository (or any parent up to mount point /)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
It seems like you downloaded `pie` instead of git-cloning it or installing it with pip.
We won't be able to check compatibility between pretrained models and `pie` version.

 - model: mymodel.tar
 - tasks: lemma
Tagging file [Lemmatisation/moliere.txt]...
100%|█████████████████████████████████████████| 146/146 [00:04<00:00, 30.27it/s]


Attention, _Pie_ n'est pas _Pie extended_. Pour ajouter le modèle à ce dernier vous pouvez suivre [le tuto](https://github.com/hipster-philology/nlp-pie-taggers#add-a-model).

Il faut désormais un modèle pour la POS – mais vous avez désormais les armes pour le faire vous-même.

In [28]:
!python --version

Python 3.8.6
